In [10]:
import sys
import os

# rootディレクトリへのパスを設定
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# codeディレクトリへのパスをsys.pathに追加
sys.path.append(os.path.join(root_dir, 'my_codes'))


In [11]:
from my_codes import fetch_articles


In [ ]:
import os

In [ ]:
os.getcwd()